In [1]:
import sys
sys.path.append('../')

import os
import json
import time
from glob import glob

from utils.elastix import excute_cmd, register_elastix, control_points_transformix
from utils.filemanager import create_directory_if_not_exists, replace_text_in_file, add_and_delete_rows, delete_added_rows
from utils.landmarks import get_landmarks_from_txt, write_landmarks_to_list
from utils.metrics import compute_TRE
from utils.utils import format_elapsed_time

# To allow auto reload to this notebook after modifying any external file imported
%load_ext autoreload
%autoreload 2

In [2]:
train_path = '../dataset/train'

# prepare the paths
exhale_volumes = [path.replace('\\', '/') for path in sorted(glob(os.path.join(train_path, "***" , "*eBHCT.nii.gz"), recursive=True))]
inhale_volumes = [path.replace('\\', '/') for path in sorted(glob(os.path.join(train_path, "***" , "*iBHCT.nii.gz"), recursive=True))]

exhale_seg = [path.replace('\\', '/') for path in sorted(glob(os.path.join(train_path, "***" , "*eBHCT_lung.nii.gz"), recursive=True))]
inhale_seg = [path.replace('\\', '/') for path in sorted(glob(os.path.join(train_path, "***" , "*iBHCT_lung.nii.gz"), recursive=True))]

exhale_volumes

['../dataset/train/copd1/copd1_eBHCT.nii.gz',
 '../dataset/train/copd2/copd2_eBHCT.nii.gz',
 '../dataset/train/copd3/copd3_eBHCT.nii.gz',
 '../dataset/train/copd4/copd4_eBHCT.nii.gz']

In [4]:
Par_base = '../elastix-parameters/Par0003'

# Setting the experiment registration parameter
params_list = [
    'Par0003.affine.txt',
    ]

reg_params      = ' '.join(['-p {}'.format(os.path.join(Par_base,param)) for param in params_list]).replace('\\', '/')
reg_params_key  = '+'.join(['{}'.format(param) for param in params_list])

print(f"Experimenting using {reg_params} params command...")
print(f"Key for the experiment: {reg_params_key}...")
print( "-----------------------------------------------------------------------------------------------------------------------------------------------")

overall_start_time = time.time()

for e_path, i_path, e_seg_path, i_seg_path in zip(exhale_volumes, inhale_volumes, exhale_seg, inhale_seg):
    # get file name
    e_filename_full = e_path.split('/')[-1].split('.')[0] #copd1_eBHCT, ..
    i_filename_full = i_path.split('/')[-1].split('.')[0] #copd1_iBHCT, ..

    sample_name = i_path.split('/')[-1].split('_')[0] #copd1, copd2, ...

    # load the dataset dictionary
    with open('../dataset/description.json', 'r') as json_file:
        dictionary = json.loads(json_file.read())
    file_information = dictionary['train'][sample_name]
    print(file_information)

    # get control points path from dataset dir
    e_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_eBH_xyz_r1.txt'
    i_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_iBH_xyz_r1.txt'

    # start the time only when the registration starts
    loop_start_time = time.time()

    # elastix registration
    print(f"Registering {e_filename_full} and {i_filename_full}...")    
    register_elastix(
        fixed_path = i_path, 
        moving_path = e_path, 
        fMask = i_seg_path,
        mMask= e_seg_path,
        reg_params = reg_params,
        reg_params_key = reg_params_key,
        create_dir_callback = create_directory_if_not_exists,
        excute_cmd_callback = excute_cmd)

    # transformix control point transformation
    print(f"Accessing the last transform parameter file TransformParameters.{len(params_list)-1}.txt")
    output_path = control_points_transformix(
        fixed_path = i_path, 
        moving_path = e_path,
        reg_params_key = reg_params_key,
        input_points = i_cntl_pt, 
        transform_path = f'output/{reg_params_key}/images/output_{i_filename_full}/{e_filename_full}/TransformParameters.{len(params_list)-1}.txt',
        replace_text_in_file_callback = replace_text_in_file,
        create_dir_callback = create_directory_if_not_exists, 
        excute_cmd_callback = excute_cmd)

    loop_end_time = time.time()
    loop_elapsed_minutes, loop_elapsed_seconds = format_elapsed_time(loop_start_time, loop_end_time)
    print(f"Time for current registration loop: {loop_elapsed_minutes} minutes and {loop_elapsed_seconds} seconds")

    # get the transformed landmarks
    landmarks_path = os.path.join(output_path, 'outputpoints.txt')
    transformed_landmarks = get_landmarks_from_txt(landmarks_path, search_key='OutputIndexFixed')

    # write the landmarks into the output directory of the points
    output_landmarks_path = os.path.join(output_path, 'outputpoints_transformed.txt')
    write_landmarks_to_list(transformed_landmarks, output_landmarks_path)
    print("Transformed landmarks saved to: ", output_landmarks_path)

    # evaluate
    # TRE_mean, TRE_std = compute_TRE(i_cntl_pt, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (Before Registration):- ", f"(Mean TRE: {file_information['displacement_mean']})", f"(STD TRE: {file_information['displacement_std']}).")
    
    TRE_mean, TRE_std = compute_TRE(output_landmarks_path, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (After Registration):- ", f"(Mean TRE: {TRE_mean})", f"(STD TRE: {TRE_std}). \n")\
    
overall_end_time = time.time()
overall_elapsed_minutes, overall_elapsed_seconds = format_elapsed_time(overall_start_time, overall_end_time)
print(f"Time for overall registrations: {overall_elapsed_minutes} minutes and {overall_elapsed_seconds} seconds")

Experimenting using -p ../elastix-parameters/Par0003/Par0003.affine.txt params command...
Key for the experiment: Par0003.affine.txt...
-----------------------------------------------------------------------------------------------------------------------------------------------
{'name': 'copd1', 'image_dim': [512, 512, 121], 'voxel_dim': [0.625, 0.625, 2.5], 'features': 773, 'displacement_mean': 25.9, 'displacement_std': 11.57, 'origin': [1, 1, 1]}
Registering copd1_eBHCT and copd1_iBHCT...
Excuting command:  elastix -f "../dataset/train/copd1/copd1_iBHCT.nii.gz" -m "../dataset/train/copd1/copd1_eBHCT.nii.gz" -fMask ../dataset/train/copd1/copd1_iBHCT_lung.nii.gz -mMask ../dataset/train/copd1/copd1_eBHCT_lung.nii.gz -p ../elastix-parameters/Par0003/Par0003.affine.txt -out "output/Par0003.affine.txt/images/output_copd1_iBHCT/copd1_eBHCT"
Accessing the last transform parameter file TransformParameters.0.txt
Time for current registration loop: 0 minutes and 30 seconds
Transformed landmark

In [5]:
Par_base = '../elastix-parameters/Par0003'

# Setting the experiment registration parameter
params_list = [
    'Par0003.bs-R1-fg.txt',
    ]

reg_params      = ' '.join(['-p {}'.format(os.path.join(Par_base,param)) for param in params_list]).replace('\\', '/')
reg_params_key  = '+'.join(['{}'.format(param) for param in params_list])

print(f"Experimenting using {reg_params} params command...")
print(f"Key for the experiment: {reg_params_key}...")
print( "-----------------------------------------------------------------------------------------------------------------------------------------------")

overall_start_time = time.time()

for e_path, i_path, e_seg_path, i_seg_path in zip(exhale_volumes, inhale_volumes, exhale_seg, inhale_seg):
    # get file name
    e_filename_full = e_path.split('/')[-1].split('.')[0] #copd1_eBHCT, ..
    i_filename_full = i_path.split('/')[-1].split('.')[0] #copd1_iBHCT, ..

    sample_name = i_path.split('/')[-1].split('_')[0] #copd1, copd2, ...

    # load the dataset dictionary
    with open('../dataset/description.json', 'r') as json_file:
        dictionary = json.loads(json_file.read())
    file_information = dictionary['train'][sample_name]
    print(file_information)

    # get control points path from dataset dir
    e_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_eBH_xyz_r1.txt'
    i_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_iBH_xyz_r1.txt'

    # start the time only when the registration starts
    loop_start_time = time.time()

    # elastix registration
    print(f"Registering {e_filename_full} and {i_filename_full}...")    
    register_elastix(
        fixed_path = i_path, 
        moving_path = e_path, 
        fMask = i_seg_path,
        mMask= e_seg_path,
        reg_params = reg_params,
        reg_params_key = reg_params_key,
        create_dir_callback = create_directory_if_not_exists,
        excute_cmd_callback = excute_cmd)

    # transformix control point transformation
    print(f"Accessing the last transform parameter file TransformParameters.{len(params_list)-1}.txt")
    output_path = control_points_transformix(
        fixed_path = i_path, 
        moving_path = e_path,
        reg_params_key = reg_params_key,
        input_points = i_cntl_pt, 
        transform_path = f'output/{reg_params_key}/images/output_{i_filename_full}/{e_filename_full}/TransformParameters.{len(params_list)-1}.txt',
        replace_text_in_file_callback = replace_text_in_file,
        create_dir_callback = create_directory_if_not_exists, 
        excute_cmd_callback = excute_cmd)

    loop_end_time = time.time()
    loop_elapsed_minutes, loop_elapsed_seconds = format_elapsed_time(loop_start_time, loop_end_time)
    print(f"Time for current registration loop: {loop_elapsed_minutes} minutes and {loop_elapsed_seconds} seconds")

    # get the transformed landmarks
    landmarks_path = os.path.join(output_path, 'outputpoints.txt')
    transformed_landmarks = get_landmarks_from_txt(landmarks_path, search_key='OutputIndexFixed')

    # write the landmarks into the output directory of the points
    output_landmarks_path = os.path.join(output_path, 'outputpoints_transformed.txt')
    write_landmarks_to_list(transformed_landmarks, output_landmarks_path)
    print("Transformed landmarks saved to: ", output_landmarks_path)

    # evaluate
    # TRE_mean, TRE_std = compute_TRE(i_cntl_pt, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (Before Registration):- ", f"(Mean TRE: {file_information['displacement_mean']})", f"(STD TRE: {file_information['displacement_std']}).")
    
    TRE_mean, TRE_std = compute_TRE(output_landmarks_path, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (After Registration):- ", f"(Mean TRE: {TRE_mean})", f"(STD TRE: {TRE_std}). \n")\
    
overall_end_time = time.time()
overall_elapsed_minutes, overall_elapsed_seconds = format_elapsed_time(overall_start_time, overall_end_time)
print(f"Time for overall registrations: {overall_elapsed_minutes} minutes and {overall_elapsed_seconds} seconds")

Experimenting using -p ../elastix-parameters/Par0003/Par0003.bs-R1-fg.txt params command...
Key for the experiment: Par0003.bs-R1-fg.txt...
-----------------------------------------------------------------------------------------------------------------------------------------------
{'name': 'copd1', 'image_dim': [512, 512, 121], 'voxel_dim': [0.625, 0.625, 2.5], 'features': 773, 'displacement_mean': 25.9, 'displacement_std': 11.57, 'origin': [1, 1, 1]}
Registering copd1_eBHCT and copd1_iBHCT...
Excuting command:  elastix -f "../dataset/train/copd1/copd1_iBHCT.nii.gz" -m "../dataset/train/copd1/copd1_eBHCT.nii.gz" -fMask ../dataset/train/copd1/copd1_iBHCT_lung.nii.gz -mMask ../dataset/train/copd1/copd1_eBHCT_lung.nii.gz -p ../elastix-parameters/Par0003/Par0003.bs-R1-fg.txt -out "output/Par0003.bs-R1-fg.txt/images/output_copd1_iBHCT/copd1_eBHCT"
Accessing the last transform parameter file TransformParameters.0.txt
Time for current registration loop: 0 minutes and 10 seconds
Transformed 

In [6]:
Par_base = '../elastix-parameters/Par0003'

# Setting the experiment registration parameter
params_list = [
    'Par0003.bs-R1-ug.txt',
    ]

reg_params      = ' '.join(['-p {}'.format(os.path.join(Par_base,param)) for param in params_list]).replace('\\', '/')
reg_params_key  = '+'.join(['{}'.format(param) for param in params_list])

print(f"Experimenting using {reg_params} params command...")
print(f"Key for the experiment: {reg_params_key}...")
print( "-----------------------------------------------------------------------------------------------------------------------------------------------")

overall_start_time = time.time()

for e_path, i_path, e_seg_path, i_seg_path in zip(exhale_volumes, inhale_volumes, exhale_seg, inhale_seg):
    # get file name
    e_filename_full = e_path.split('/')[-1].split('.')[0] #copd1_eBHCT, ..
    i_filename_full = i_path.split('/')[-1].split('.')[0] #copd1_iBHCT, ..

    sample_name = i_path.split('/')[-1].split('_')[0] #copd1, copd2, ...

    # load the dataset dictionary
    with open('../dataset/description.json', 'r') as json_file:
        dictionary = json.loads(json_file.read())
    file_information = dictionary['train'][sample_name]
    print(file_information)

    # get control points path from dataset dir
    e_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_eBH_xyz_r1.txt'
    i_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_iBH_xyz_r1.txt'

    # start the time only when the registration starts
    loop_start_time = time.time()

    # elastix registration
    print(f"Registering {e_filename_full} and {i_filename_full}...")    
    register_elastix(
        fixed_path = i_path, 
        moving_path = e_path, 
        fMask = i_seg_path,
        mMask= e_seg_path,
        reg_params = reg_params,
        reg_params_key = reg_params_key,
        create_dir_callback = create_directory_if_not_exists,
        excute_cmd_callback = excute_cmd)

    # transformix control point transformation
    print(f"Accessing the last transform parameter file TransformParameters.{len(params_list)-1}.txt")
    output_path = control_points_transformix(
        fixed_path = i_path, 
        moving_path = e_path,
        reg_params_key = reg_params_key,
        input_points = i_cntl_pt, 
        transform_path = f'output/{reg_params_key}/images/output_{i_filename_full}/{e_filename_full}/TransformParameters.{len(params_list)-1}.txt',
        replace_text_in_file_callback = replace_text_in_file,
        create_dir_callback = create_directory_if_not_exists, 
        excute_cmd_callback = excute_cmd)

    loop_end_time = time.time()
    loop_elapsed_minutes, loop_elapsed_seconds = format_elapsed_time(loop_start_time, loop_end_time)
    print(f"Time for current registration loop: {loop_elapsed_minutes} minutes and {loop_elapsed_seconds} seconds")

    # get the transformed landmarks
    landmarks_path = os.path.join(output_path, 'outputpoints.txt')
    transformed_landmarks = get_landmarks_from_txt(landmarks_path, search_key='OutputIndexFixed')

    # write the landmarks into the output directory of the points
    output_landmarks_path = os.path.join(output_path, 'outputpoints_transformed.txt')
    write_landmarks_to_list(transformed_landmarks, output_landmarks_path)
    print("Transformed landmarks saved to: ", output_landmarks_path)

    # evaluate
    # TRE_mean, TRE_std = compute_TRE(i_cntl_pt, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (Before Registration):- ", f"(Mean TRE: {file_information['displacement_mean']})", f"(STD TRE: {file_information['displacement_std']}).")
    
    TRE_mean, TRE_std = compute_TRE(output_landmarks_path, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (After Registration):- ", f"(Mean TRE: {TRE_mean})", f"(STD TRE: {TRE_std}). \n")\
    
overall_end_time = time.time()
overall_elapsed_minutes, overall_elapsed_seconds = format_elapsed_time(overall_start_time, overall_end_time)
print(f"Time for overall registrations: {overall_elapsed_minutes} minutes and {overall_elapsed_seconds} seconds")

Experimenting using -p ../elastix-parameters/Par0003/Par0003.bs-R1-ug.txt params command...
Key for the experiment: Par0003.bs-R1-ug.txt...
-----------------------------------------------------------------------------------------------------------------------------------------------
{'name': 'copd1', 'image_dim': [512, 512, 121], 'voxel_dim': [0.625, 0.625, 2.5], 'features': 773, 'displacement_mean': 25.9, 'displacement_std': 11.57, 'origin': [1, 1, 1]}
Registering copd1_eBHCT and copd1_iBHCT...
Excuting command:  elastix -f "../dataset/train/copd1/copd1_iBHCT.nii.gz" -m "../dataset/train/copd1/copd1_eBHCT.nii.gz" -fMask ../dataset/train/copd1/copd1_iBHCT_lung.nii.gz -mMask ../dataset/train/copd1/copd1_eBHCT_lung.nii.gz -p ../elastix-parameters/Par0003/Par0003.bs-R1-ug.txt -out "output/Par0003.bs-R1-ug.txt/images/output_copd1_iBHCT/copd1_eBHCT"
Accessing the last transform parameter file TransformParameters.0.txt
Time for current registration loop: 0 minutes and 9 seconds
Transformed l

In [7]:
Par_base = '../elastix-parameters/Par0003'

# Setting the experiment registration parameter
params_list = [
    'Par0003.bs-R2-fg.txt',
    ]

reg_params      = ' '.join(['-p {}'.format(os.path.join(Par_base,param)) for param in params_list]).replace('\\', '/')
reg_params_key  = '+'.join(['{}'.format(param) for param in params_list])

print(f"Experimenting using {reg_params} params command...")
print(f"Key for the experiment: {reg_params_key}...")
print( "-----------------------------------------------------------------------------------------------------------------------------------------------")

overall_start_time = time.time()

for e_path, i_path, e_seg_path, i_seg_path in zip(exhale_volumes, inhale_volumes, exhale_seg, inhale_seg):
    # get file name
    e_filename_full = e_path.split('/')[-1].split('.')[0] #copd1_eBHCT, ..
    i_filename_full = i_path.split('/')[-1].split('.')[0] #copd1_iBHCT, ..

    sample_name = i_path.split('/')[-1].split('_')[0] #copd1, copd2, ...

    # load the dataset dictionary
    with open('../dataset/description.json', 'r') as json_file:
        dictionary = json.loads(json_file.read())
    file_information = dictionary['train'][sample_name]
    print(file_information)

    # get control points path from dataset dir
    e_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_eBH_xyz_r1.txt'
    i_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_iBH_xyz_r1.txt'

    # start the time only when the registration starts
    loop_start_time = time.time()

    # elastix registration
    print(f"Registering {e_filename_full} and {i_filename_full}...")    
    register_elastix(
        fixed_path = i_path, 
        moving_path = e_path, 
        fMask = i_seg_path,
        mMask= e_seg_path,
        reg_params = reg_params,
        reg_params_key = reg_params_key,
        create_dir_callback = create_directory_if_not_exists,
        excute_cmd_callback = excute_cmd)

    # transformix control point transformation
    print(f"Accessing the last transform parameter file TransformParameters.{len(params_list)-1}.txt")
    output_path = control_points_transformix(
        fixed_path = i_path, 
        moving_path = e_path,
        reg_params_key = reg_params_key,
        input_points = i_cntl_pt, 
        transform_path = f'output/{reg_params_key}/images/output_{i_filename_full}/{e_filename_full}/TransformParameters.{len(params_list)-1}.txt',
        replace_text_in_file_callback = replace_text_in_file,
        create_dir_callback = create_directory_if_not_exists, 
        excute_cmd_callback = excute_cmd)

    loop_end_time = time.time()
    loop_elapsed_minutes, loop_elapsed_seconds = format_elapsed_time(loop_start_time, loop_end_time)
    print(f"Time for current registration loop: {loop_elapsed_minutes} minutes and {loop_elapsed_seconds} seconds")

    # get the transformed landmarks
    landmarks_path = os.path.join(output_path, 'outputpoints.txt')
    transformed_landmarks = get_landmarks_from_txt(landmarks_path, search_key='OutputIndexFixed')

    # write the landmarks into the output directory of the points
    output_landmarks_path = os.path.join(output_path, 'outputpoints_transformed.txt')
    write_landmarks_to_list(transformed_landmarks, output_landmarks_path)
    print("Transformed landmarks saved to: ", output_landmarks_path)

    # evaluate
    # TRE_mean, TRE_std = compute_TRE(i_cntl_pt, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (Before Registration):- ", f"(Mean TRE: {file_information['displacement_mean']})", f"(STD TRE: {file_information['displacement_std']}).")
    
    TRE_mean, TRE_std = compute_TRE(output_landmarks_path, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (After Registration):- ", f"(Mean TRE: {TRE_mean})", f"(STD TRE: {TRE_std}). \n")\
    
overall_end_time = time.time()
overall_elapsed_minutes, overall_elapsed_seconds = format_elapsed_time(overall_start_time, overall_end_time)
print(f"Time for overall registrations: {overall_elapsed_minutes} minutes and {overall_elapsed_seconds} seconds")

Experimenting using -p ../elastix-parameters/Par0003/Par0003.bs-R2-fg.txt params command...
Key for the experiment: Par0003.bs-R2-fg.txt...
-----------------------------------------------------------------------------------------------------------------------------------------------
{'name': 'copd1', 'image_dim': [512, 512, 121], 'voxel_dim': [0.625, 0.625, 2.5], 'features': 773, 'displacement_mean': 25.9, 'displacement_std': 11.57, 'origin': [1, 1, 1]}
Registering copd1_eBHCT and copd1_iBHCT...
Excuting command:  elastix -f "../dataset/train/copd1/copd1_iBHCT.nii.gz" -m "../dataset/train/copd1/copd1_eBHCT.nii.gz" -fMask ../dataset/train/copd1/copd1_iBHCT_lung.nii.gz -mMask ../dataset/train/copd1/copd1_eBHCT_lung.nii.gz -p ../elastix-parameters/Par0003/Par0003.bs-R2-fg.txt -out "output/Par0003.bs-R2-fg.txt/images/output_copd1_iBHCT/copd1_eBHCT"
Accessing the last transform parameter file TransformParameters.0.txt
Time for current registration loop: 0 minutes and 19 seconds
Transformed 

In [8]:
Par_base = '../elastix-parameters/Par0003'

# Setting the experiment registration parameter
params_list = [
    'Par0003.bs-R2-ug.txt',
    ]

reg_params      = ' '.join(['-p {}'.format(os.path.join(Par_base,param)) for param in params_list]).replace('\\', '/')
reg_params_key  = '+'.join(['{}'.format(param) for param in params_list])

print(f"Experimenting using {reg_params} params command...")
print(f"Key for the experiment: {reg_params_key}...")
print( "-----------------------------------------------------------------------------------------------------------------------------------------------")

overall_start_time = time.time()

for e_path, i_path, e_seg_path, i_seg_path in zip(exhale_volumes, inhale_volumes, exhale_seg, inhale_seg):
    # get file name
    e_filename_full = e_path.split('/')[-1].split('.')[0] #copd1_eBHCT, ..
    i_filename_full = i_path.split('/')[-1].split('.')[0] #copd1_iBHCT, ..

    sample_name = i_path.split('/')[-1].split('_')[0] #copd1, copd2, ...

    # load the dataset dictionary
    with open('../dataset/description.json', 'r') as json_file:
        dictionary = json.loads(json_file.read())
    file_information = dictionary['train'][sample_name]
    print(file_information)

    # get control points path from dataset dir
    e_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_eBH_xyz_r1.txt'
    i_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_iBH_xyz_r1.txt'

    # start the time only when the registration starts
    loop_start_time = time.time()

    # elastix registration
    print(f"Registering {e_filename_full} and {i_filename_full}...")    
    register_elastix(
        fixed_path = i_path, 
        moving_path = e_path, 
        fMask = i_seg_path,
        mMask= e_seg_path,
        reg_params = reg_params,
        reg_params_key = reg_params_key,
        create_dir_callback = create_directory_if_not_exists,
        excute_cmd_callback = excute_cmd)

    # transformix control point transformation
    print(f"Accessing the last transform parameter file TransformParameters.{len(params_list)-1}.txt")
    output_path = control_points_transformix(
        fixed_path = i_path, 
        moving_path = e_path,
        reg_params_key = reg_params_key,
        input_points = i_cntl_pt, 
        transform_path = f'output/{reg_params_key}/images/output_{i_filename_full}/{e_filename_full}/TransformParameters.{len(params_list)-1}.txt',
        replace_text_in_file_callback = replace_text_in_file,
        create_dir_callback = create_directory_if_not_exists, 
        excute_cmd_callback = excute_cmd)

    loop_end_time = time.time()
    loop_elapsed_minutes, loop_elapsed_seconds = format_elapsed_time(loop_start_time, loop_end_time)
    print(f"Time for current registration loop: {loop_elapsed_minutes} minutes and {loop_elapsed_seconds} seconds")

    # get the transformed landmarks
    landmarks_path = os.path.join(output_path, 'outputpoints.txt')
    transformed_landmarks = get_landmarks_from_txt(landmarks_path, search_key='OutputIndexFixed')

    # write the landmarks into the output directory of the points
    output_landmarks_path = os.path.join(output_path, 'outputpoints_transformed.txt')
    write_landmarks_to_list(transformed_landmarks, output_landmarks_path)
    print("Transformed landmarks saved to: ", output_landmarks_path)

    # evaluate
    # TRE_mean, TRE_std = compute_TRE(i_cntl_pt, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (Before Registration):- ", f"(Mean TRE: {file_information['displacement_mean']})", f"(STD TRE: {file_information['displacement_std']}).")
    
    TRE_mean, TRE_std = compute_TRE(output_landmarks_path, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (After Registration):- ", f"(Mean TRE: {TRE_mean})", f"(STD TRE: {TRE_std}). \n")\
    
overall_end_time = time.time()
overall_elapsed_minutes, overall_elapsed_seconds = format_elapsed_time(overall_start_time, overall_end_time)
print(f"Time for overall registrations: {overall_elapsed_minutes} minutes and {overall_elapsed_seconds} seconds")

Experimenting using -p ../elastix-parameters/Par0003/Par0003.bs-R2-ug.txt params command...
Key for the experiment: Par0003.bs-R2-ug.txt...
-----------------------------------------------------------------------------------------------------------------------------------------------
{'name': 'copd1', 'image_dim': [512, 512, 121], 'voxel_dim': [0.625, 0.625, 2.5], 'features': 773, 'displacement_mean': 25.9, 'displacement_std': 11.57, 'origin': [1, 1, 1]}
Registering copd1_eBHCT and copd1_iBHCT...
Excuting command:  elastix -f "../dataset/train/copd1/copd1_iBHCT.nii.gz" -m "../dataset/train/copd1/copd1_eBHCT.nii.gz" -fMask ../dataset/train/copd1/copd1_iBHCT_lung.nii.gz -mMask ../dataset/train/copd1/copd1_eBHCT_lung.nii.gz -p ../elastix-parameters/Par0003/Par0003.bs-R2-ug.txt -out "output/Par0003.bs-R2-ug.txt/images/output_copd1_iBHCT/copd1_eBHCT"
Accessing the last transform parameter file TransformParameters.0.txt
Time for current registration loop: 0 minutes and 17 seconds
Transformed 

In [9]:
Par_base = '../elastix-parameters/Par0003'

# Setting the experiment registration parameter
params_list = [
    'Par0003.bs-R3-fg.txt',
    ]

reg_params      = ' '.join(['-p {}'.format(os.path.join(Par_base,param)) for param in params_list]).replace('\\', '/')
reg_params_key  = '+'.join(['{}'.format(param) for param in params_list])

print(f"Experimenting using {reg_params} params command...")
print(f"Key for the experiment: {reg_params_key}...")
print( "-----------------------------------------------------------------------------------------------------------------------------------------------")

overall_start_time = time.time()

for e_path, i_path, e_seg_path, i_seg_path in zip(exhale_volumes, inhale_volumes, exhale_seg, inhale_seg):
    # get file name
    e_filename_full = e_path.split('/')[-1].split('.')[0] #copd1_eBHCT, ..
    i_filename_full = i_path.split('/')[-1].split('.')[0] #copd1_iBHCT, ..

    sample_name = i_path.split('/')[-1].split('_')[0] #copd1, copd2, ...

    # load the dataset dictionary
    with open('../dataset/description.json', 'r') as json_file:
        dictionary = json.loads(json_file.read())
    file_information = dictionary['train'][sample_name]
    print(file_information)

    # get control points path from dataset dir
    e_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_eBH_xyz_r1.txt'
    i_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_iBH_xyz_r1.txt'

    # start the time only when the registration starts
    loop_start_time = time.time()

    # elastix registration
    print(f"Registering {e_filename_full} and {i_filename_full}...")    
    register_elastix(
        fixed_path = i_path, 
        moving_path = e_path, 
        fMask = i_seg_path,
        mMask= e_seg_path,
        reg_params = reg_params,
        reg_params_key = reg_params_key,
        create_dir_callback = create_directory_if_not_exists,
        excute_cmd_callback = excute_cmd)

    # transformix control point transformation
    print(f"Accessing the last transform parameter file TransformParameters.{len(params_list)-1}.txt")
    output_path = control_points_transformix(
        fixed_path = i_path, 
        moving_path = e_path,
        reg_params_key = reg_params_key,
        input_points = i_cntl_pt, 
        transform_path = f'output/{reg_params_key}/images/output_{i_filename_full}/{e_filename_full}/TransformParameters.{len(params_list)-1}.txt',
        replace_text_in_file_callback = replace_text_in_file,
        create_dir_callback = create_directory_if_not_exists, 
        excute_cmd_callback = excute_cmd)

    loop_end_time = time.time()
    loop_elapsed_minutes, loop_elapsed_seconds = format_elapsed_time(loop_start_time, loop_end_time)
    print(f"Time for current registration loop: {loop_elapsed_minutes} minutes and {loop_elapsed_seconds} seconds")

    # get the transformed landmarks
    landmarks_path = os.path.join(output_path, 'outputpoints.txt')
    transformed_landmarks = get_landmarks_from_txt(landmarks_path, search_key='OutputIndexFixed')

    # write the landmarks into the output directory of the points
    output_landmarks_path = os.path.join(output_path, 'outputpoints_transformed.txt')
    write_landmarks_to_list(transformed_landmarks, output_landmarks_path)
    print("Transformed landmarks saved to: ", output_landmarks_path)

    # evaluate
    # TRE_mean, TRE_std = compute_TRE(i_cntl_pt, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (Before Registration):- ", f"(Mean TRE: {file_information['displacement_mean']})", f"(STD TRE: {file_information['displacement_std']}).")
    
    TRE_mean, TRE_std = compute_TRE(output_landmarks_path, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (After Registration):- ", f"(Mean TRE: {TRE_mean})", f"(STD TRE: {TRE_std}). \n")\
    
overall_end_time = time.time()
overall_elapsed_minutes, overall_elapsed_seconds = format_elapsed_time(overall_start_time, overall_end_time)
print(f"Time for overall registrations: {overall_elapsed_minutes} minutes and {overall_elapsed_seconds} seconds")

Experimenting using -p ../elastix-parameters/Par0003/Par0003.bs-R3-fg.txt params command...
Key for the experiment: Par0003.bs-R3-fg.txt...
-----------------------------------------------------------------------------------------------------------------------------------------------
{'name': 'copd1', 'image_dim': [512, 512, 121], 'voxel_dim': [0.625, 0.625, 2.5], 'features': 773, 'displacement_mean': 25.9, 'displacement_std': 11.57, 'origin': [1, 1, 1]}
Registering copd1_eBHCT and copd1_iBHCT...
Excuting command:  elastix -f "../dataset/train/copd1/copd1_iBHCT.nii.gz" -m "../dataset/train/copd1/copd1_eBHCT.nii.gz" -fMask ../dataset/train/copd1/copd1_iBHCT_lung.nii.gz -mMask ../dataset/train/copd1/copd1_eBHCT_lung.nii.gz -p ../elastix-parameters/Par0003/Par0003.bs-R3-fg.txt -out "output/Par0003.bs-R3-fg.txt/images/output_copd1_iBHCT/copd1_eBHCT"
Accessing the last transform parameter file TransformParameters.0.txt
Time for current registration loop: 0 minutes and 26 seconds
Transformed 

In [10]:
Par_base = '../elastix-parameters/Par0003'

# Setting the experiment registration parameter
params_list = [
    'Par0003.bs-R3-ug.txt',
    ]

reg_params      = ' '.join(['-p {}'.format(os.path.join(Par_base,param)) for param in params_list]).replace('\\', '/')
reg_params_key  = '+'.join(['{}'.format(param) for param in params_list])

print(f"Experimenting using {reg_params} params command...")
print(f"Key for the experiment: {reg_params_key}...")
print( "-----------------------------------------------------------------------------------------------------------------------------------------------")

overall_start_time = time.time()

for e_path, i_path, e_seg_path, i_seg_path in zip(exhale_volumes, inhale_volumes, exhale_seg, inhale_seg):
    # get file name
    e_filename_full = e_path.split('/')[-1].split('.')[0] #copd1_eBHCT, ..
    i_filename_full = i_path.split('/')[-1].split('.')[0] #copd1_iBHCT, ..

    sample_name = i_path.split('/')[-1].split('_')[0] #copd1, copd2, ...

    # load the dataset dictionary
    with open('../dataset/description.json', 'r') as json_file:
        dictionary = json.loads(json_file.read())
    file_information = dictionary['train'][sample_name]
    print(file_information)

    # get control points path from dataset dir
    e_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_eBH_xyz_r1.txt'
    i_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_iBH_xyz_r1.txt'

    # start the time only when the registration starts
    loop_start_time = time.time()

    # elastix registration
    print(f"Registering {e_filename_full} and {i_filename_full}...")    
    register_elastix(
        fixed_path = i_path, 
        moving_path = e_path, 
        fMask = i_seg_path,
        mMask= e_seg_path,
        reg_params = reg_params,
        reg_params_key = reg_params_key,
        create_dir_callback = create_directory_if_not_exists,
        excute_cmd_callback = excute_cmd)

    # transformix control point transformation
    print(f"Accessing the last transform parameter file TransformParameters.{len(params_list)-1}.txt")
    output_path = control_points_transformix(
        fixed_path = i_path, 
        moving_path = e_path,
        reg_params_key = reg_params_key,
        input_points = i_cntl_pt, 
        transform_path = f'output/{reg_params_key}/images/output_{i_filename_full}/{e_filename_full}/TransformParameters.{len(params_list)-1}.txt',
        replace_text_in_file_callback = replace_text_in_file,
        create_dir_callback = create_directory_if_not_exists, 
        excute_cmd_callback = excute_cmd)

    loop_end_time = time.time()
    loop_elapsed_minutes, loop_elapsed_seconds = format_elapsed_time(loop_start_time, loop_end_time)
    print(f"Time for current registration loop: {loop_elapsed_minutes} minutes and {loop_elapsed_seconds} seconds")

    # get the transformed landmarks
    landmarks_path = os.path.join(output_path, 'outputpoints.txt')
    transformed_landmarks = get_landmarks_from_txt(landmarks_path, search_key='OutputIndexFixed')

    # write the landmarks into the output directory of the points
    output_landmarks_path = os.path.join(output_path, 'outputpoints_transformed.txt')
    write_landmarks_to_list(transformed_landmarks, output_landmarks_path)
    print("Transformed landmarks saved to: ", output_landmarks_path)

    # evaluate
    # TRE_mean, TRE_std = compute_TRE(i_cntl_pt, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (Before Registration):- ", f"(Mean TRE: {file_information['displacement_mean']})", f"(STD TRE: {file_information['displacement_std']}).")
    
    TRE_mean, TRE_std = compute_TRE(output_landmarks_path, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (After Registration):- ", f"(Mean TRE: {TRE_mean})", f"(STD TRE: {TRE_std}). \n")\
    
overall_end_time = time.time()
overall_elapsed_minutes, overall_elapsed_seconds = format_elapsed_time(overall_start_time, overall_end_time)
print(f"Time for overall registrations: {overall_elapsed_minutes} minutes and {overall_elapsed_seconds} seconds")

Experimenting using -p ../elastix-parameters/Par0003/Par0003.bs-R3-ug.txt params command...
Key for the experiment: Par0003.bs-R3-ug.txt...
-----------------------------------------------------------------------------------------------------------------------------------------------
{'name': 'copd1', 'image_dim': [512, 512, 121], 'voxel_dim': [0.625, 0.625, 2.5], 'features': 773, 'displacement_mean': 25.9, 'displacement_std': 11.57, 'origin': [1, 1, 1]}
Registering copd1_eBHCT and copd1_iBHCT...
Excuting command:  elastix -f "../dataset/train/copd1/copd1_iBHCT.nii.gz" -m "../dataset/train/copd1/copd1_eBHCT.nii.gz" -fMask ../dataset/train/copd1/copd1_iBHCT_lung.nii.gz -mMask ../dataset/train/copd1/copd1_eBHCT_lung.nii.gz -p ../elastix-parameters/Par0003/Par0003.bs-R3-ug.txt -out "output/Par0003.bs-R3-ug.txt/images/output_copd1_iBHCT/copd1_eBHCT"
Accessing the last transform parameter file TransformParameters.0.txt
Time for current registration loop: 0 minutes and 24 seconds
Transformed 

In [11]:
Par_base = '../elastix-parameters/Par0003'

# Setting the experiment registration parameter
params_list = [
    'Par0003.bs-R4-fg.txt',
    ]

reg_params      = ' '.join(['-p {}'.format(os.path.join(Par_base,param)) for param in params_list]).replace('\\', '/')
reg_params_key  = '+'.join(['{}'.format(param) for param in params_list])

print(f"Experimenting using {reg_params} params command...")
print(f"Key for the experiment: {reg_params_key}...")
print( "-----------------------------------------------------------------------------------------------------------------------------------------------")

overall_start_time = time.time()

for e_path, i_path, e_seg_path, i_seg_path in zip(exhale_volumes, inhale_volumes, exhale_seg, inhale_seg):
    # get file name
    e_filename_full = e_path.split('/')[-1].split('.')[0] #copd1_eBHCT, ..
    i_filename_full = i_path.split('/')[-1].split('.')[0] #copd1_iBHCT, ..

    sample_name = i_path.split('/')[-1].split('_')[0] #copd1, copd2, ...

    # load the dataset dictionary
    with open('../dataset/description.json', 'r') as json_file:
        dictionary = json.loads(json_file.read())
    file_information = dictionary['train'][sample_name]
    print(file_information)

    # get control points path from dataset dir
    e_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_eBH_xyz_r1.txt'
    i_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_iBH_xyz_r1.txt'

    # start the time only when the registration starts
    loop_start_time = time.time()

    # elastix registration
    print(f"Registering {e_filename_full} and {i_filename_full}...")    
    register_elastix(
        fixed_path = i_path, 
        moving_path = e_path, 
        fMask = i_seg_path,
        mMask= e_seg_path,
        reg_params = reg_params,
        reg_params_key = reg_params_key,
        create_dir_callback = create_directory_if_not_exists,
        excute_cmd_callback = excute_cmd)

    # transformix control point transformation
    print(f"Accessing the last transform parameter file TransformParameters.{len(params_list)-1}.txt")
    output_path = control_points_transformix(
        fixed_path = i_path, 
        moving_path = e_path,
        reg_params_key = reg_params_key,
        input_points = i_cntl_pt, 
        transform_path = f'output/{reg_params_key}/images/output_{i_filename_full}/{e_filename_full}/TransformParameters.{len(params_list)-1}.txt',
        replace_text_in_file_callback = replace_text_in_file,
        create_dir_callback = create_directory_if_not_exists, 
        excute_cmd_callback = excute_cmd)

    loop_end_time = time.time()
    loop_elapsed_minutes, loop_elapsed_seconds = format_elapsed_time(loop_start_time, loop_end_time)
    print(f"Time for current registration loop: {loop_elapsed_minutes} minutes and {loop_elapsed_seconds} seconds")

    # get the transformed landmarks
    landmarks_path = os.path.join(output_path, 'outputpoints.txt')
    transformed_landmarks = get_landmarks_from_txt(landmarks_path, search_key='OutputIndexFixed')

    # write the landmarks into the output directory of the points
    output_landmarks_path = os.path.join(output_path, 'outputpoints_transformed.txt')
    write_landmarks_to_list(transformed_landmarks, output_landmarks_path)
    print("Transformed landmarks saved to: ", output_landmarks_path)

    # evaluate
    # TRE_mean, TRE_std = compute_TRE(i_cntl_pt, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (Before Registration):- ", f"(Mean TRE: {file_information['displacement_mean']})", f"(STD TRE: {file_information['displacement_std']}).")
    
    TRE_mean, TRE_std = compute_TRE(output_landmarks_path, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (After Registration):- ", f"(Mean TRE: {TRE_mean})", f"(STD TRE: {TRE_std}). \n")\
    
overall_end_time = time.time()
overall_elapsed_minutes, overall_elapsed_seconds = format_elapsed_time(overall_start_time, overall_end_time)
print(f"Time for overall registrations: {overall_elapsed_minutes} minutes and {overall_elapsed_seconds} seconds")

Experimenting using -p ../elastix-parameters/Par0003/Par0003.bs-R4-fg.txt params command...
Key for the experiment: Par0003.bs-R4-fg.txt...
-----------------------------------------------------------------------------------------------------------------------------------------------
{'name': 'copd1', 'image_dim': [512, 512, 121], 'voxel_dim': [0.625, 0.625, 2.5], 'features': 773, 'displacement_mean': 25.9, 'displacement_std': 11.57, 'origin': [1, 1, 1]}
Registering copd1_eBHCT and copd1_iBHCT...
Excuting command:  elastix -f "../dataset/train/copd1/copd1_iBHCT.nii.gz" -m "../dataset/train/copd1/copd1_eBHCT.nii.gz" -fMask ../dataset/train/copd1/copd1_iBHCT_lung.nii.gz -mMask ../dataset/train/copd1/copd1_eBHCT_lung.nii.gz -p ../elastix-parameters/Par0003/Par0003.bs-R4-fg.txt -out "output/Par0003.bs-R4-fg.txt/images/output_copd1_iBHCT/copd1_eBHCT"
Accessing the last transform parameter file TransformParameters.0.txt
Time for current registration loop: 0 minutes and 34 seconds
Transformed 

In [12]:
Par_base = '../elastix-parameters/Par0003'

# Setting the experiment registration parameter
params_list = [
    'Par0003.bs-R4-ug.txt',
    ]

reg_params      = ' '.join(['-p {}'.format(os.path.join(Par_base,param)) for param in params_list]).replace('\\', '/')
reg_params_key  = '+'.join(['{}'.format(param) for param in params_list])

print(f"Experimenting using {reg_params} params command...")
print(f"Key for the experiment: {reg_params_key}...")
print( "-----------------------------------------------------------------------------------------------------------------------------------------------")

overall_start_time = time.time()

for e_path, i_path, e_seg_path, i_seg_path in zip(exhale_volumes, inhale_volumes, exhale_seg, inhale_seg):
    # get file name
    e_filename_full = e_path.split('/')[-1].split('.')[0] #copd1_eBHCT, ..
    i_filename_full = i_path.split('/')[-1].split('.')[0] #copd1_iBHCT, ..

    sample_name = i_path.split('/')[-1].split('_')[0] #copd1, copd2, ...

    # load the dataset dictionary
    with open('../dataset/description.json', 'r') as json_file:
        dictionary = json.loads(json_file.read())
    file_information = dictionary['train'][sample_name]
    print(file_information)

    # get control points path from dataset dir
    e_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_eBH_xyz_r1.txt'
    i_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_iBH_xyz_r1.txt'

    # start the time only when the registration starts
    loop_start_time = time.time()

    # elastix registration
    print(f"Registering {e_filename_full} and {i_filename_full}...")    
    register_elastix(
        fixed_path = i_path, 
        moving_path = e_path, 
        fMask = i_seg_path,
        mMask= e_seg_path,
        reg_params = reg_params,
        reg_params_key = reg_params_key,
        create_dir_callback = create_directory_if_not_exists,
        excute_cmd_callback = excute_cmd)

    # transformix control point transformation
    print(f"Accessing the last transform parameter file TransformParameters.{len(params_list)-1}.txt")
    output_path = control_points_transformix(
        fixed_path = i_path, 
        moving_path = e_path,
        reg_params_key = reg_params_key,
        input_points = i_cntl_pt, 
        transform_path = f'output/{reg_params_key}/images/output_{i_filename_full}/{e_filename_full}/TransformParameters.{len(params_list)-1}.txt',
        replace_text_in_file_callback = replace_text_in_file,
        create_dir_callback = create_directory_if_not_exists, 
        excute_cmd_callback = excute_cmd)

    loop_end_time = time.time()
    loop_elapsed_minutes, loop_elapsed_seconds = format_elapsed_time(loop_start_time, loop_end_time)
    print(f"Time for current registration loop: {loop_elapsed_minutes} minutes and {loop_elapsed_seconds} seconds")

    # get the transformed landmarks
    landmarks_path = os.path.join(output_path, 'outputpoints.txt')
    transformed_landmarks = get_landmarks_from_txt(landmarks_path, search_key='OutputIndexFixed')

    # write the landmarks into the output directory of the points
    output_landmarks_path = os.path.join(output_path, 'outputpoints_transformed.txt')
    write_landmarks_to_list(transformed_landmarks, output_landmarks_path)
    print("Transformed landmarks saved to: ", output_landmarks_path)

    # evaluate
    # TRE_mean, TRE_std = compute_TRE(i_cntl_pt, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (Before Registration):- ", f"(Mean TRE: {file_information['displacement_mean']})", f"(STD TRE: {file_information['displacement_std']}).")
    
    TRE_mean, TRE_std = compute_TRE(output_landmarks_path, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (After Registration):- ", f"(Mean TRE: {TRE_mean})", f"(STD TRE: {TRE_std}). \n")\
    
overall_end_time = time.time()
overall_elapsed_minutes, overall_elapsed_seconds = format_elapsed_time(overall_start_time, overall_end_time)
print(f"Time for overall registrations: {overall_elapsed_minutes} minutes and {overall_elapsed_seconds} seconds")

Experimenting using -p ../elastix-parameters/Par0003/Par0003.bs-R4-ug.txt params command...
Key for the experiment: Par0003.bs-R4-ug.txt...
-----------------------------------------------------------------------------------------------------------------------------------------------
{'name': 'copd1', 'image_dim': [512, 512, 121], 'voxel_dim': [0.625, 0.625, 2.5], 'features': 773, 'displacement_mean': 25.9, 'displacement_std': 11.57, 'origin': [1, 1, 1]}
Registering copd1_eBHCT and copd1_iBHCT...
Excuting command:  elastix -f "../dataset/train/copd1/copd1_iBHCT.nii.gz" -m "../dataset/train/copd1/copd1_eBHCT.nii.gz" -fMask ../dataset/train/copd1/copd1_iBHCT_lung.nii.gz -mMask ../dataset/train/copd1/copd1_eBHCT_lung.nii.gz -p ../elastix-parameters/Par0003/Par0003.bs-R4-ug.txt -out "output/Par0003.bs-R4-ug.txt/images/output_copd1_iBHCT/copd1_eBHCT"
Accessing the last transform parameter file TransformParameters.0.txt
Time for current registration loop: 0 minutes and 30 seconds
Transformed 

In [13]:
Par_base = '../elastix-parameters/Par0003'

# Setting the experiment registration parameter
params_list = [
    'Par0003.bs-R5-fg.txt',
    ]

reg_params      = ' '.join(['-p {}'.format(os.path.join(Par_base,param)) for param in params_list]).replace('\\', '/')
reg_params_key  = '+'.join(['{}'.format(param) for param in params_list])

print(f"Experimenting using {reg_params} params command...")
print(f"Key for the experiment: {reg_params_key}...")
print( "-----------------------------------------------------------------------------------------------------------------------------------------------")

overall_start_time = time.time()

for e_path, i_path, e_seg_path, i_seg_path in zip(exhale_volumes, inhale_volumes, exhale_seg, inhale_seg):
    # get file name
    e_filename_full = e_path.split('/')[-1].split('.')[0] #copd1_eBHCT, ..
    i_filename_full = i_path.split('/')[-1].split('.')[0] #copd1_iBHCT, ..

    sample_name = i_path.split('/')[-1].split('_')[0] #copd1, copd2, ...

    # load the dataset dictionary
    with open('../dataset/description.json', 'r') as json_file:
        dictionary = json.loads(json_file.read())
    file_information = dictionary['train'][sample_name]
    print(file_information)

    # get control points path from dataset dir
    e_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_eBH_xyz_r1.txt'
    i_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_iBH_xyz_r1.txt'

    # start the time only when the registration starts
    loop_start_time = time.time()

    # elastix registration
    print(f"Registering {e_filename_full} and {i_filename_full}...")    
    register_elastix(
        fixed_path = i_path, 
        moving_path = e_path, 
        fMask = i_seg_path,
        mMask= e_seg_path,
        reg_params = reg_params,
        reg_params_key = reg_params_key,
        create_dir_callback = create_directory_if_not_exists,
        excute_cmd_callback = excute_cmd)

    # transformix control point transformation
    print(f"Accessing the last transform parameter file TransformParameters.{len(params_list)-1}.txt")
    output_path = control_points_transformix(
        fixed_path = i_path, 
        moving_path = e_path,
        reg_params_key = reg_params_key,
        input_points = i_cntl_pt, 
        transform_path = f'output/{reg_params_key}/images/output_{i_filename_full}/{e_filename_full}/TransformParameters.{len(params_list)-1}.txt',
        replace_text_in_file_callback = replace_text_in_file,
        create_dir_callback = create_directory_if_not_exists, 
        excute_cmd_callback = excute_cmd)

    loop_end_time = time.time()
    loop_elapsed_minutes, loop_elapsed_seconds = format_elapsed_time(loop_start_time, loop_end_time)
    print(f"Time for current registration loop: {loop_elapsed_minutes} minutes and {loop_elapsed_seconds} seconds")

    # get the transformed landmarks
    landmarks_path = os.path.join(output_path, 'outputpoints.txt')
    transformed_landmarks = get_landmarks_from_txt(landmarks_path, search_key='OutputIndexFixed')

    # write the landmarks into the output directory of the points
    output_landmarks_path = os.path.join(output_path, 'outputpoints_transformed.txt')
    write_landmarks_to_list(transformed_landmarks, output_landmarks_path)
    print("Transformed landmarks saved to: ", output_landmarks_path)

    # evaluate
    # TRE_mean, TRE_std = compute_TRE(i_cntl_pt, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (Before Registration):- ", f"(Mean TRE: {file_information['displacement_mean']})", f"(STD TRE: {file_information['displacement_std']}).")
    
    TRE_mean, TRE_std = compute_TRE(output_landmarks_path, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (After Registration):- ", f"(Mean TRE: {TRE_mean})", f"(STD TRE: {TRE_std}). \n")\
    
overall_end_time = time.time()
overall_elapsed_minutes, overall_elapsed_seconds = format_elapsed_time(overall_start_time, overall_end_time)
print(f"Time for overall registrations: {overall_elapsed_minutes} minutes and {overall_elapsed_seconds} seconds")

Experimenting using -p ../elastix-parameters/Par0003/Par0003.bs-R5-fg.txt params command...
Key for the experiment: Par0003.bs-R5-fg.txt...
-----------------------------------------------------------------------------------------------------------------------------------------------
{'name': 'copd1', 'image_dim': [512, 512, 121], 'voxel_dim': [0.625, 0.625, 2.5], 'features': 773, 'displacement_mean': 25.9, 'displacement_std': 11.57, 'origin': [1, 1, 1]}
Registering copd1_eBHCT and copd1_iBHCT...
Excuting command:  elastix -f "../dataset/train/copd1/copd1_iBHCT.nii.gz" -m "../dataset/train/copd1/copd1_eBHCT.nii.gz" -fMask ../dataset/train/copd1/copd1_iBHCT_lung.nii.gz -mMask ../dataset/train/copd1/copd1_eBHCT_lung.nii.gz -p ../elastix-parameters/Par0003/Par0003.bs-R5-fg.txt -out "output/Par0003.bs-R5-fg.txt/images/output_copd1_iBHCT/copd1_eBHCT"
Accessing the last transform parameter file TransformParameters.0.txt
Time for current registration loop: 0 minutes and 39 seconds
Transformed 

In [14]:
Par_base = '../elastix-parameters/Par0003'

# Setting the experiment registration parameter
params_list = [
    'Par0003.bs-R5-ug.txt',
    ]

reg_params      = ' '.join(['-p {}'.format(os.path.join(Par_base,param)) for param in params_list]).replace('\\', '/')
reg_params_key  = '+'.join(['{}'.format(param) for param in params_list])

print(f"Experimenting using {reg_params} params command...")
print(f"Key for the experiment: {reg_params_key}...")
print( "-----------------------------------------------------------------------------------------------------------------------------------------------")

overall_start_time = time.time()

for e_path, i_path, e_seg_path, i_seg_path in zip(exhale_volumes, inhale_volumes, exhale_seg, inhale_seg):
    # get file name
    e_filename_full = e_path.split('/')[-1].split('.')[0] #copd1_eBHCT, ..
    i_filename_full = i_path.split('/')[-1].split('.')[0] #copd1_iBHCT, ..

    sample_name = i_path.split('/')[-1].split('_')[0] #copd1, copd2, ...

    # load the dataset dictionary
    with open('../dataset/description.json', 'r') as json_file:
        dictionary = json.loads(json_file.read())
    file_information = dictionary['train'][sample_name]
    print(file_information)

    # get control points path from dataset dir
    e_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_eBH_xyz_r1.txt'
    i_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_iBH_xyz_r1.txt'

    # start the time only when the registration starts
    loop_start_time = time.time()

    # elastix registration
    print(f"Registering {e_filename_full} and {i_filename_full}...")    
    register_elastix(
        fixed_path = i_path, 
        moving_path = e_path, 
        fMask = i_seg_path,
        mMask= e_seg_path,
        reg_params = reg_params,
        reg_params_key = reg_params_key,
        create_dir_callback = create_directory_if_not_exists,
        excute_cmd_callback = excute_cmd)

    # transformix control point transformation
    print(f"Accessing the last transform parameter file TransformParameters.{len(params_list)-1}.txt")
    output_path = control_points_transformix(
        fixed_path = i_path, 
        moving_path = e_path,
        reg_params_key = reg_params_key,
        input_points = i_cntl_pt, 
        transform_path = f'output/{reg_params_key}/images/output_{i_filename_full}/{e_filename_full}/TransformParameters.{len(params_list)-1}.txt',
        replace_text_in_file_callback = replace_text_in_file,
        create_dir_callback = create_directory_if_not_exists, 
        excute_cmd_callback = excute_cmd)

    loop_end_time = time.time()
    loop_elapsed_minutes, loop_elapsed_seconds = format_elapsed_time(loop_start_time, loop_end_time)
    print(f"Time for current registration loop: {loop_elapsed_minutes} minutes and {loop_elapsed_seconds} seconds")

    # get the transformed landmarks
    landmarks_path = os.path.join(output_path, 'outputpoints.txt')
    transformed_landmarks = get_landmarks_from_txt(landmarks_path, search_key='OutputIndexFixed')

    # write the landmarks into the output directory of the points
    output_landmarks_path = os.path.join(output_path, 'outputpoints_transformed.txt')
    write_landmarks_to_list(transformed_landmarks, output_landmarks_path)
    print("Transformed landmarks saved to: ", output_landmarks_path)

    # evaluate
    # TRE_mean, TRE_std = compute_TRE(i_cntl_pt, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (Before Registration):- ", f"(Mean TRE: {file_information['displacement_mean']})", f"(STD TRE: {file_information['displacement_std']}).")
    
    TRE_mean, TRE_std = compute_TRE(output_landmarks_path, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (After Registration):- ", f"(Mean TRE: {TRE_mean})", f"(STD TRE: {TRE_std}). \n")\
    
overall_end_time = time.time()
overall_elapsed_minutes, overall_elapsed_seconds = format_elapsed_time(overall_start_time, overall_end_time)
print(f"Time for overall registrations: {overall_elapsed_minutes} minutes and {overall_elapsed_seconds} seconds")

Experimenting using -p ../elastix-parameters/Par0003/Par0003.bs-R5-ug.txt params command...
Key for the experiment: Par0003.bs-R5-ug.txt...
-----------------------------------------------------------------------------------------------------------------------------------------------
{'name': 'copd1', 'image_dim': [512, 512, 121], 'voxel_dim': [0.625, 0.625, 2.5], 'features': 773, 'displacement_mean': 25.9, 'displacement_std': 11.57, 'origin': [1, 1, 1]}
Registering copd1_eBHCT and copd1_iBHCT...
Excuting command:  elastix -f "../dataset/train/copd1/copd1_iBHCT.nii.gz" -m "../dataset/train/copd1/copd1_eBHCT.nii.gz" -fMask ../dataset/train/copd1/copd1_iBHCT_lung.nii.gz -mMask ../dataset/train/copd1/copd1_eBHCT_lung.nii.gz -p ../elastix-parameters/Par0003/Par0003.bs-R5-ug.txt -out "output/Par0003.bs-R5-ug.txt/images/output_copd1_iBHCT/copd1_eBHCT"
Accessing the last transform parameter file TransformParameters.0.txt
Time for current registration loop: 0 minutes and 40 seconds
Transformed 

In [15]:
Par_base = '../elastix-parameters/Par0003'

# Setting the experiment registration parameter
params_list = [
    'Par0003.bs-R6-fg.txt',
    ]

reg_params      = ' '.join(['-p {}'.format(os.path.join(Par_base,param)) for param in params_list]).replace('\\', '/')
reg_params_key  = '+'.join(['{}'.format(param) for param in params_list])

print(f"Experimenting using {reg_params} params command...")
print(f"Key for the experiment: {reg_params_key}...")
print( "-----------------------------------------------------------------------------------------------------------------------------------------------")

overall_start_time = time.time()

for e_path, i_path, e_seg_path, i_seg_path in zip(exhale_volumes, inhale_volumes, exhale_seg, inhale_seg):
    # get file name
    e_filename_full = e_path.split('/')[-1].split('.')[0] #copd1_eBHCT, ..
    i_filename_full = i_path.split('/')[-1].split('.')[0] #copd1_iBHCT, ..

    sample_name = i_path.split('/')[-1].split('_')[0] #copd1, copd2, ...

    # load the dataset dictionary
    with open('../dataset/description.json', 'r') as json_file:
        dictionary = json.loads(json_file.read())
    file_information = dictionary['train'][sample_name]
    print(file_information)

    # get control points path from dataset dir
    e_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_eBH_xyz_r1.txt'
    i_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_iBH_xyz_r1.txt'

    # start the time only when the registration starts
    loop_start_time = time.time()

    # elastix registration
    print(f"Registering {e_filename_full} and {i_filename_full}...")    
    register_elastix(
        fixed_path = i_path, 
        moving_path = e_path, 
        fMask = i_seg_path,
        mMask= e_seg_path,
        reg_params = reg_params,
        reg_params_key = reg_params_key,
        create_dir_callback = create_directory_if_not_exists,
        excute_cmd_callback = excute_cmd)

    # transformix control point transformation
    print(f"Accessing the last transform parameter file TransformParameters.{len(params_list)-1}.txt")
    output_path = control_points_transformix(
        fixed_path = i_path, 
        moving_path = e_path,
        reg_params_key = reg_params_key,
        input_points = i_cntl_pt, 
        transform_path = f'output/{reg_params_key}/images/output_{i_filename_full}/{e_filename_full}/TransformParameters.{len(params_list)-1}.txt',
        replace_text_in_file_callback = replace_text_in_file,
        create_dir_callback = create_directory_if_not_exists, 
        excute_cmd_callback = excute_cmd)

    loop_end_time = time.time()
    loop_elapsed_minutes, loop_elapsed_seconds = format_elapsed_time(loop_start_time, loop_end_time)
    print(f"Time for current registration loop: {loop_elapsed_minutes} minutes and {loop_elapsed_seconds} seconds")

    # get the transformed landmarks
    landmarks_path = os.path.join(output_path, 'outputpoints.txt')
    transformed_landmarks = get_landmarks_from_txt(landmarks_path, search_key='OutputIndexFixed')

    # write the landmarks into the output directory of the points
    output_landmarks_path = os.path.join(output_path, 'outputpoints_transformed.txt')
    write_landmarks_to_list(transformed_landmarks, output_landmarks_path)
    print("Transformed landmarks saved to: ", output_landmarks_path)

    # evaluate
    # TRE_mean, TRE_std = compute_TRE(i_cntl_pt, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (Before Registration):- ", f"(Mean TRE: {file_information['displacement_mean']})", f"(STD TRE: {file_information['displacement_std']}).")
    
    TRE_mean, TRE_std = compute_TRE(output_landmarks_path, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (After Registration):- ", f"(Mean TRE: {TRE_mean})", f"(STD TRE: {TRE_std}). \n")\
    
overall_end_time = time.time()
overall_elapsed_minutes, overall_elapsed_seconds = format_elapsed_time(overall_start_time, overall_end_time)
print(f"Time for overall registrations: {overall_elapsed_minutes} minutes and {overall_elapsed_seconds} seconds")

Experimenting using -p ../elastix-parameters/Par0003/Par0003.bs-R6-fg.txt params command...
Key for the experiment: Par0003.bs-R6-fg.txt...
-----------------------------------------------------------------------------------------------------------------------------------------------
{'name': 'copd1', 'image_dim': [512, 512, 121], 'voxel_dim': [0.625, 0.625, 2.5], 'features': 773, 'displacement_mean': 25.9, 'displacement_std': 11.57, 'origin': [1, 1, 1]}
Registering copd1_eBHCT and copd1_iBHCT...
Excuting command:  elastix -f "../dataset/train/copd1/copd1_iBHCT.nii.gz" -m "../dataset/train/copd1/copd1_eBHCT.nii.gz" -fMask ../dataset/train/copd1/copd1_iBHCT_lung.nii.gz -mMask ../dataset/train/copd1/copd1_eBHCT_lung.nii.gz -p ../elastix-parameters/Par0003/Par0003.bs-R6-fg.txt -out "output/Par0003.bs-R6-fg.txt/images/output_copd1_iBHCT/copd1_eBHCT"
Accessing the last transform parameter file TransformParameters.0.txt
Time for current registration loop: 0 minutes and 54 seconds
Transformed 

In [16]:
Par_base = '../elastix-parameters/Par0003'

# Setting the experiment registration parameter
params_list = [
    'Par0003.bs-R6-ug.txt',
    ]

reg_params      = ' '.join(['-p {}'.format(os.path.join(Par_base,param)) for param in params_list]).replace('\\', '/')
reg_params_key  = '+'.join(['{}'.format(param) for param in params_list])

print(f"Experimenting using {reg_params} params command...")
print(f"Key for the experiment: {reg_params_key}...")
print( "-----------------------------------------------------------------------------------------------------------------------------------------------")

overall_start_time = time.time()

for e_path, i_path, e_seg_path, i_seg_path in zip(exhale_volumes, inhale_volumes, exhale_seg, inhale_seg):
    # get file name
    e_filename_full = e_path.split('/')[-1].split('.')[0] #copd1_eBHCT, ..
    i_filename_full = i_path.split('/')[-1].split('.')[0] #copd1_iBHCT, ..

    sample_name = i_path.split('/')[-1].split('_')[0] #copd1, copd2, ...

    # load the dataset dictionary
    with open('../dataset/description.json', 'r') as json_file:
        dictionary = json.loads(json_file.read())
    file_information = dictionary['train'][sample_name]
    print(file_information)

    # get control points path from dataset dir
    e_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_eBH_xyz_r1.txt'
    i_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_iBH_xyz_r1.txt'

    # start the time only when the registration starts
    loop_start_time = time.time()

    # elastix registration
    print(f"Registering {e_filename_full} and {i_filename_full}...")    
    register_elastix(
        fixed_path = i_path, 
        moving_path = e_path, 
        fMask = i_seg_path,
        mMask= e_seg_path,
        reg_params = reg_params,
        reg_params_key = reg_params_key,
        create_dir_callback = create_directory_if_not_exists,
        excute_cmd_callback = excute_cmd)

    # transformix control point transformation
    print(f"Accessing the last transform parameter file TransformParameters.{len(params_list)-1}.txt")
    output_path = control_points_transformix(
        fixed_path = i_path, 
        moving_path = e_path,
        reg_params_key = reg_params_key,
        input_points = i_cntl_pt, 
        transform_path = f'output/{reg_params_key}/images/output_{i_filename_full}/{e_filename_full}/TransformParameters.{len(params_list)-1}.txt',
        replace_text_in_file_callback = replace_text_in_file,
        create_dir_callback = create_directory_if_not_exists, 
        excute_cmd_callback = excute_cmd)

    loop_end_time = time.time()
    loop_elapsed_minutes, loop_elapsed_seconds = format_elapsed_time(loop_start_time, loop_end_time)
    print(f"Time for current registration loop: {loop_elapsed_minutes} minutes and {loop_elapsed_seconds} seconds")

    # get the transformed landmarks
    landmarks_path = os.path.join(output_path, 'outputpoints.txt')
    transformed_landmarks = get_landmarks_from_txt(landmarks_path, search_key='OutputIndexFixed')

    # write the landmarks into the output directory of the points
    output_landmarks_path = os.path.join(output_path, 'outputpoints_transformed.txt')
    write_landmarks_to_list(transformed_landmarks, output_landmarks_path)
    print("Transformed landmarks saved to: ", output_landmarks_path)

    # evaluate
    # TRE_mean, TRE_std = compute_TRE(i_cntl_pt, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (Before Registration):- ", f"(Mean TRE: {file_information['displacement_mean']})", f"(STD TRE: {file_information['displacement_std']}).")
    
    TRE_mean, TRE_std = compute_TRE(output_landmarks_path, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (After Registration):- ", f"(Mean TRE: {TRE_mean})", f"(STD TRE: {TRE_std}). \n")\
    
overall_end_time = time.time()
overall_elapsed_minutes, overall_elapsed_seconds = format_elapsed_time(overall_start_time, overall_end_time)
print(f"Time for overall registrations: {overall_elapsed_minutes} minutes and {overall_elapsed_seconds} seconds")

Experimenting using -p ../elastix-parameters/Par0003/Par0003.bs-R6-ug.txt params command...
Key for the experiment: Par0003.bs-R6-ug.txt...
-----------------------------------------------------------------------------------------------------------------------------------------------
{'name': 'copd1', 'image_dim': [512, 512, 121], 'voxel_dim': [0.625, 0.625, 2.5], 'features': 773, 'displacement_mean': 25.9, 'displacement_std': 11.57, 'origin': [1, 1, 1]}
Registering copd1_eBHCT and copd1_iBHCT...
Excuting command:  elastix -f "../dataset/train/copd1/copd1_iBHCT.nii.gz" -m "../dataset/train/copd1/copd1_eBHCT.nii.gz" -fMask ../dataset/train/copd1/copd1_iBHCT_lung.nii.gz -mMask ../dataset/train/copd1/copd1_eBHCT_lung.nii.gz -p ../elastix-parameters/Par0003/Par0003.bs-R6-ug.txt -out "output/Par0003.bs-R6-ug.txt/images/output_copd1_iBHCT/copd1_eBHCT"
Accessing the last transform parameter file TransformParameters.0.txt
Time for current registration loop: 0 minutes and 46 seconds
Transformed 

In [17]:
Par_base = '../elastix-parameters/Par0003'

# Setting the experiment registration parameter
params_list = [
    'Par0003.bs-R7-fg.txt',
    ]

reg_params      = ' '.join(['-p {}'.format(os.path.join(Par_base,param)) for param in params_list]).replace('\\', '/')
reg_params_key  = '+'.join(['{}'.format(param) for param in params_list])

print(f"Experimenting using {reg_params} params command...")
print(f"Key for the experiment: {reg_params_key}...")
print( "-----------------------------------------------------------------------------------------------------------------------------------------------")

overall_start_time = time.time()

for e_path, i_path, e_seg_path, i_seg_path in zip(exhale_volumes, inhale_volumes, exhale_seg, inhale_seg):
    # get file name
    e_filename_full = e_path.split('/')[-1].split('.')[0] #copd1_eBHCT, ..
    i_filename_full = i_path.split('/')[-1].split('.')[0] #copd1_iBHCT, ..

    sample_name = i_path.split('/')[-1].split('_')[0] #copd1, copd2, ...

    # load the dataset dictionary
    with open('../dataset/description.json', 'r') as json_file:
        dictionary = json.loads(json_file.read())
    file_information = dictionary['train'][sample_name]
    print(file_information)

    # get control points path from dataset dir
    e_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_eBH_xyz_r1.txt'
    i_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_iBH_xyz_r1.txt'

    # start the time only when the registration starts
    loop_start_time = time.time()

    # elastix registration
    print(f"Registering {e_filename_full} and {i_filename_full}...")    
    register_elastix(
        fixed_path = i_path, 
        moving_path = e_path, 
        fMask = i_seg_path,
        mMask= e_seg_path,
        reg_params = reg_params,
        reg_params_key = reg_params_key,
        create_dir_callback = create_directory_if_not_exists,
        excute_cmd_callback = excute_cmd)

    # transformix control point transformation
    print(f"Accessing the last transform parameter file TransformParameters.{len(params_list)-1}.txt")
    output_path = control_points_transformix(
        fixed_path = i_path, 
        moving_path = e_path,
        reg_params_key = reg_params_key,
        input_points = i_cntl_pt, 
        transform_path = f'output/{reg_params_key}/images/output_{i_filename_full}/{e_filename_full}/TransformParameters.{len(params_list)-1}.txt',
        replace_text_in_file_callback = replace_text_in_file,
        create_dir_callback = create_directory_if_not_exists, 
        excute_cmd_callback = excute_cmd)

    loop_end_time = time.time()
    loop_elapsed_minutes, loop_elapsed_seconds = format_elapsed_time(loop_start_time, loop_end_time)
    print(f"Time for current registration loop: {loop_elapsed_minutes} minutes and {loop_elapsed_seconds} seconds")

    # get the transformed landmarks
    landmarks_path = os.path.join(output_path, 'outputpoints.txt')
    transformed_landmarks = get_landmarks_from_txt(landmarks_path, search_key='OutputIndexFixed')

    # write the landmarks into the output directory of the points
    output_landmarks_path = os.path.join(output_path, 'outputpoints_transformed.txt')
    write_landmarks_to_list(transformed_landmarks, output_landmarks_path)
    print("Transformed landmarks saved to: ", output_landmarks_path)

    # evaluate
    # TRE_mean, TRE_std = compute_TRE(i_cntl_pt, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (Before Registration):- ", f"(Mean TRE: {file_information['displacement_mean']})", f"(STD TRE: {file_information['displacement_std']}).")
    
    TRE_mean, TRE_std = compute_TRE(output_landmarks_path, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (After Registration):- ", f"(Mean TRE: {TRE_mean})", f"(STD TRE: {TRE_std}). \n")\
    
overall_end_time = time.time()
overall_elapsed_minutes, overall_elapsed_seconds = format_elapsed_time(overall_start_time, overall_end_time)
print(f"Time for overall registrations: {overall_elapsed_minutes} minutes and {overall_elapsed_seconds} seconds")

Experimenting using -p ../elastix-parameters/Par0003/Par0003.bs-R7-fg.txt params command...
Key for the experiment: Par0003.bs-R7-fg.txt...
-----------------------------------------------------------------------------------------------------------------------------------------------
{'name': 'copd1', 'image_dim': [512, 512, 121], 'voxel_dim': [0.625, 0.625, 2.5], 'features': 773, 'displacement_mean': 25.9, 'displacement_std': 11.57, 'origin': [1, 1, 1]}
Registering copd1_eBHCT and copd1_iBHCT...
Excuting command:  elastix -f "../dataset/train/copd1/copd1_iBHCT.nii.gz" -m "../dataset/train/copd1/copd1_eBHCT.nii.gz" -fMask ../dataset/train/copd1/copd1_iBHCT_lung.nii.gz -mMask ../dataset/train/copd1/copd1_eBHCT_lung.nii.gz -p ../elastix-parameters/Par0003/Par0003.bs-R7-fg.txt -out "output/Par0003.bs-R7-fg.txt/images/output_copd1_iBHCT/copd1_eBHCT"
Accessing the last transform parameter file TransformParameters.0.txt
Time for current registration loop: 0 minutes and 55 seconds
Transformed 

In [18]:
Par_base = '../elastix-parameters/Par0003'

# Setting the experiment registration parameter
params_list = [
    'Par0003.bs-R7-ug.txt',
    ]

reg_params      = ' '.join(['-p {}'.format(os.path.join(Par_base,param)) for param in params_list]).replace('\\', '/')
reg_params_key  = '+'.join(['{}'.format(param) for param in params_list])

print(f"Experimenting using {reg_params} params command...")
print(f"Key for the experiment: {reg_params_key}...")
print( "-----------------------------------------------------------------------------------------------------------------------------------------------")

overall_start_time = time.time()

for e_path, i_path, e_seg_path, i_seg_path in zip(exhale_volumes, inhale_volumes, exhale_seg, inhale_seg):
    # get file name
    e_filename_full = e_path.split('/')[-1].split('.')[0] #copd1_eBHCT, ..
    i_filename_full = i_path.split('/')[-1].split('.')[0] #copd1_iBHCT, ..

    sample_name = i_path.split('/')[-1].split('_')[0] #copd1, copd2, ...

    # load the dataset dictionary
    with open('../dataset/description.json', 'r') as json_file:
        dictionary = json.loads(json_file.read())
    file_information = dictionary['train'][sample_name]
    print(file_information)

    # get control points path from dataset dir
    e_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_eBH_xyz_r1.txt'
    i_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_iBH_xyz_r1.txt'

    # start the time only when the registration starts
    loop_start_time = time.time()

    # elastix registration
    print(f"Registering {e_filename_full} and {i_filename_full}...")    
    register_elastix(
        fixed_path = i_path, 
        moving_path = e_path, 
        fMask = i_seg_path,
        mMask= e_seg_path,
        reg_params = reg_params,
        reg_params_key = reg_params_key,
        create_dir_callback = create_directory_if_not_exists,
        excute_cmd_callback = excute_cmd)

    # transformix control point transformation
    print(f"Accessing the last transform parameter file TransformParameters.{len(params_list)-1}.txt")
    output_path = control_points_transformix(
        fixed_path = i_path, 
        moving_path = e_path,
        reg_params_key = reg_params_key,
        input_points = i_cntl_pt, 
        transform_path = f'output/{reg_params_key}/images/output_{i_filename_full}/{e_filename_full}/TransformParameters.{len(params_list)-1}.txt',
        replace_text_in_file_callback = replace_text_in_file,
        create_dir_callback = create_directory_if_not_exists, 
        excute_cmd_callback = excute_cmd)

    loop_end_time = time.time()
    loop_elapsed_minutes, loop_elapsed_seconds = format_elapsed_time(loop_start_time, loop_end_time)
    print(f"Time for current registration loop: {loop_elapsed_minutes} minutes and {loop_elapsed_seconds} seconds")

    # get the transformed landmarks
    landmarks_path = os.path.join(output_path, 'outputpoints.txt')
    transformed_landmarks = get_landmarks_from_txt(landmarks_path, search_key='OutputIndexFixed')

    # write the landmarks into the output directory of the points
    output_landmarks_path = os.path.join(output_path, 'outputpoints_transformed.txt')
    write_landmarks_to_list(transformed_landmarks, output_landmarks_path)
    print("Transformed landmarks saved to: ", output_landmarks_path)

    # evaluate
    # TRE_mean, TRE_std = compute_TRE(i_cntl_pt, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (Before Registration):- ", f"(Mean TRE: {file_information['displacement_mean']})", f"(STD TRE: {file_information['displacement_std']}).")
    
    TRE_mean, TRE_std = compute_TRE(output_landmarks_path, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (After Registration):- ", f"(Mean TRE: {TRE_mean})", f"(STD TRE: {TRE_std}). \n")\
    
overall_end_time = time.time()
overall_elapsed_minutes, overall_elapsed_seconds = format_elapsed_time(overall_start_time, overall_end_time)
print(f"Time for overall registrations: {overall_elapsed_minutes} minutes and {overall_elapsed_seconds} seconds")

Experimenting using -p ../elastix-parameters/Par0003/Par0003.bs-R7-ug.txt params command...
Key for the experiment: Par0003.bs-R7-ug.txt...
-----------------------------------------------------------------------------------------------------------------------------------------------
{'name': 'copd1', 'image_dim': [512, 512, 121], 'voxel_dim': [0.625, 0.625, 2.5], 'features': 773, 'displacement_mean': 25.9, 'displacement_std': 11.57, 'origin': [1, 1, 1]}
Registering copd1_eBHCT and copd1_iBHCT...
Excuting command:  elastix -f "../dataset/train/copd1/copd1_iBHCT.nii.gz" -m "../dataset/train/copd1/copd1_eBHCT.nii.gz" -fMask ../dataset/train/copd1/copd1_iBHCT_lung.nii.gz -mMask ../dataset/train/copd1/copd1_eBHCT_lung.nii.gz -p ../elastix-parameters/Par0003/Par0003.bs-R7-ug.txt -out "output/Par0003.bs-R7-ug.txt/images/output_copd1_iBHCT/copd1_eBHCT"
Accessing the last transform parameter file TransformParameters.0.txt
Time for current registration loop: 0 minutes and 51 seconds
Transformed 

In [19]:
Par_base = '../elastix-parameters/Par0003'

# Setting the experiment registration parameter
params_list = [
    'Par0003.bs-R8-fg.txt',
    ]

reg_params      = ' '.join(['-p {}'.format(os.path.join(Par_base,param)) for param in params_list]).replace('\\', '/')
reg_params_key  = '+'.join(['{}'.format(param) for param in params_list])

print(f"Experimenting using {reg_params} params command...")
print(f"Key for the experiment: {reg_params_key}...")
print( "-----------------------------------------------------------------------------------------------------------------------------------------------")

overall_start_time = time.time()

for e_path, i_path, e_seg_path, i_seg_path in zip(exhale_volumes, inhale_volumes, exhale_seg, inhale_seg):
    # get file name
    e_filename_full = e_path.split('/')[-1].split('.')[0] #copd1_eBHCT, ..
    i_filename_full = i_path.split('/')[-1].split('.')[0] #copd1_iBHCT, ..

    sample_name = i_path.split('/')[-1].split('_')[0] #copd1, copd2, ...

    # load the dataset dictionary
    with open('../dataset/description.json', 'r') as json_file:
        dictionary = json.loads(json_file.read())
    file_information = dictionary['train'][sample_name]
    print(file_information)

    # get control points path from dataset dir
    e_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_eBH_xyz_r1.txt'
    i_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_iBH_xyz_r1.txt'

    # start the time only when the registration starts
    loop_start_time = time.time()

    # elastix registration
    print(f"Registering {e_filename_full} and {i_filename_full}...")    
    register_elastix(
        fixed_path = i_path, 
        moving_path = e_path, 
        fMask = i_seg_path,
        mMask= e_seg_path,
        reg_params = reg_params,
        reg_params_key = reg_params_key,
        create_dir_callback = create_directory_if_not_exists,
        excute_cmd_callback = excute_cmd)

    # transformix control point transformation
    print(f"Accessing the last transform parameter file TransformParameters.{len(params_list)-1}.txt")
    output_path = control_points_transformix(
        fixed_path = i_path, 
        moving_path = e_path,
        reg_params_key = reg_params_key,
        input_points = i_cntl_pt, 
        transform_path = f'output/{reg_params_key}/images/output_{i_filename_full}/{e_filename_full}/TransformParameters.{len(params_list)-1}.txt',
        replace_text_in_file_callback = replace_text_in_file,
        create_dir_callback = create_directory_if_not_exists, 
        excute_cmd_callback = excute_cmd)

    loop_end_time = time.time()
    loop_elapsed_minutes, loop_elapsed_seconds = format_elapsed_time(loop_start_time, loop_end_time)
    print(f"Time for current registration loop: {loop_elapsed_minutes} minutes and {loop_elapsed_seconds} seconds")

    # get the transformed landmarks
    landmarks_path = os.path.join(output_path, 'outputpoints.txt')
    transformed_landmarks = get_landmarks_from_txt(landmarks_path, search_key='OutputIndexFixed')

    # write the landmarks into the output directory of the points
    output_landmarks_path = os.path.join(output_path, 'outputpoints_transformed.txt')
    write_landmarks_to_list(transformed_landmarks, output_landmarks_path)
    print("Transformed landmarks saved to: ", output_landmarks_path)

    # evaluate
    # TRE_mean, TRE_std = compute_TRE(i_cntl_pt, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (Before Registration):- ", f"(Mean TRE: {file_information['displacement_mean']})", f"(STD TRE: {file_information['displacement_std']}).")
    
    TRE_mean, TRE_std = compute_TRE(output_landmarks_path, e_cntl_pt, tuple(file_information['voxel_dim']))
    print("TRE (After Registration):- ", f"(Mean TRE: {TRE_mean})", f"(STD TRE: {TRE_std}). \n")\
    
overall_end_time = time.time()
overall_elapsed_minutes, overall_elapsed_seconds = format_elapsed_time(overall_start_time, overall_end_time)
print(f"Time for overall registrations: {overall_elapsed_minutes} minutes and {overall_elapsed_seconds} seconds")

Experimenting using -p ../elastix-parameters/Par0003/Par0003.bs-R8-fg.txt params command...
Key for the experiment: Par0003.bs-R8-fg.txt...
-----------------------------------------------------------------------------------------------------------------------------------------------
{'name': 'copd1', 'image_dim': [512, 512, 121], 'voxel_dim': [0.625, 0.625, 2.5], 'features': 773, 'displacement_mean': 25.9, 'displacement_std': 11.57, 'origin': [1, 1, 1]}
Registering copd1_eBHCT and copd1_iBHCT...
Excuting command:  elastix -f "../dataset/train/copd1/copd1_iBHCT.nii.gz" -m "../dataset/train/copd1/copd1_eBHCT.nii.gz" -fMask ../dataset/train/copd1/copd1_iBHCT_lung.nii.gz -mMask ../dataset/train/copd1/copd1_eBHCT_lung.nii.gz -p ../elastix-parameters/Par0003/Par0003.bs-R8-fg.txt -out "output/Par0003.bs-R8-fg.txt/images/output_copd1_iBHCT/copd1_eBHCT"
Accessing the last transform parameter file TransformParameters.0.txt
Time for current registration loop: 1 minutes and 3 seconds
Transformed l

In [24]:
# Par_base = '../elastix-parameters/Par0003'

# # Setting the experiment registration parameter
# params_list = [
#     'Par0003.bs-R8-ug.txt',
#     ]

# reg_params      = ' '.join(['-p {}'.format(os.path.join(Par_base,param)) for param in params_list]).replace('\\', '/')
# reg_params_key  = '+'.join(['{}'.format(param) for param in params_list])

# print(f"Experimenting using {reg_params} params command...")
# print(f"Key for the experiment: {reg_params_key}...")
# print( "-----------------------------------------------------------------------------------------------------------------------------------------------")

# overall_start_time = time.time()

# for e_path, i_path, e_seg_path, i_seg_path in zip(exhale_volumes, inhale_volumes, exhale_seg, inhale_seg):
#     # get file name
#     e_filename_full = e_path.split('/')[-1].split('.')[0] #copd1_eBHCT, ..
#     i_filename_full = i_path.split('/')[-1].split('.')[0] #copd1_iBHCT, ..

#     sample_name = i_path.split('/')[-1].split('_')[0] #copd1, copd2, ...

#     # load the dataset dictionary
#     with open('../dataset/description.json', 'r') as json_file:
#         dictionary = json.loads(json_file.read())
#     file_information = dictionary['train'][sample_name]
#     print(file_information)

#     # get control points path from dataset dir
#     e_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_eBH_xyz_r1.txt'
#     i_cntl_pt = f'../dataset/train/{sample_name}/{sample_name}_300_iBH_xyz_r1.txt'

#     # start the time only when the registration starts
#     loop_start_time = time.time()

#     # elastix registration
#     print(f"Registering {e_filename_full} and {i_filename_full}...")    
#     register_elastix(
#         fixed_path = i_path, 
#         moving_path = e_path, 
#         fMask = i_seg_path,
#         mMask= e_seg_path,
#         reg_params = reg_params,
#         reg_params_key = reg_params_key,
#         create_dir_callback = create_directory_if_not_exists,
#         excute_cmd_callback = excute_cmd)

#     # transformix control point transformation
#     print(f"Accessing the last transform parameter file TransformParameters.{len(params_list)-1}.txt")
#     output_path = control_points_transformix(
#         fixed_path = i_path, 
#         moving_path = e_path,
#         reg_params_key = reg_params_key,
#         input_points = i_cntl_pt, 
#         transform_path = f'output/{reg_params_key}/images/output_{i_filename_full}/{e_filename_full}/TransformParameters.{len(params_list)-1}.txt',
#         replace_text_in_file_callback = replace_text_in_file,
#         create_dir_callback = create_directory_if_not_exists, 
#         excute_cmd_callback = excute_cmd)

#     loop_end_time = time.time()
#     loop_elapsed_minutes, loop_elapsed_seconds = format_elapsed_time(loop_start_time, loop_end_time)
#     print(f"Time for current registration loop: {loop_elapsed_minutes} minutes and {loop_elapsed_seconds} seconds")

#     # get the transformed landmarks
#     landmarks_path = os.path.join(output_path, 'outputpoints.txt')
#     transformed_landmarks = get_landmarks_from_txt(landmarks_path, search_key='OutputIndexFixed')

#     # write the landmarks into the output directory of the points
#     output_landmarks_path = os.path.join(output_path, 'outputpoints_transformed.txt')
#     write_landmarks_to_list(transformed_landmarks, output_landmarks_path)
#     print("Transformed landmarks saved to: ", output_landmarks_path)

#     # evaluate
#     # TRE_mean, TRE_std = compute_TRE(i_cntl_pt, e_cntl_pt, tuple(file_information['voxel_dim']))
#     print("TRE (Before Registration):- ", f"(Mean TRE: {file_information['displacement_mean']})", f"(STD TRE: {file_information['displacement_std']}).")
    
#     TRE_mean, TRE_std = compute_TRE(output_landmarks_path, e_cntl_pt, tuple(file_information['voxel_dim']))
#     print("TRE (After Registration):- ", f"(Mean TRE: {TRE_mean})", f"(STD TRE: {TRE_std}). \n")\
    
# overall_end_time = time.time()
# overall_elapsed_minutes, overall_elapsed_seconds = format_elapsed_time(overall_start_time, overall_end_time)
# print(f"Time for overall registrations: {overall_elapsed_minutes} minutes and {overall_elapsed_seconds} seconds")

# command fails: Description: itk::ERROR: AdvancedMattesMutualInformationMetric(0000000002B00790): Too many samples map outside moving image buffer: 499 / 2000